In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.model_selection import train_test_split as tts
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score
from catboost.utils import get_roc_curve, select_threshold
import optuna
from optuna.samplers import TPESampler
import catboost

In [4]:
df = pd.read_csv("../data/wb_school_task_2.csv")

In [15]:
REGEX = re.compile("[А-Яа-яA-z]+")
MORPH_ANALYZER = MorphAnalyzer()

def words_only(text: str, regex=REGEX) -> list:
    ''' '''
    try:
        return regex.findall(text.lower())
    except:
        return []

def lemmatize_word(token: str, pymorphy=MORPH_ANALYZER) -> str:
    return pymorphy.parse(token)[0].normal_form

def clean_text(text: str) -> list:
    tokens = words_only(text)
    lemmas = [lemmatize_word(w) for w in tokens]
    return lemmas

In [16]:
lemmas = list(tqdm(map(clean_text, df['text']), total=len(df)))

100%|██████████████████████████████████████| 3123/3123 [00:11<00:00, 275.83it/s]


In [17]:
lemmas

[['хотеть',
  'купить',
  'на',
  'замена',
  'старый',
  'по',
  'итог',
  'этот',
  'ужаснотупой',
  'не',
  'удобный',
  'рвать',
  'кутикула',
  'а',
  'не',
  'резать'],
 ['запах',
  'по',
  'сравнение',
  'с',
  'обычный',
  'dove',
  'никакой',
  'оно',
  'не',
  'парфюмировать',
  'только',
  'баночка',
  'симпатичный',
  'и',
  'вс'],
 ['кабель', 'подош', 'л', 'хорошо', 'работать'],
 ['восторг',
  'очень',
  'приятный',
  'аромат',
  'держаться',
  'долго',
  'очень',
  'удобный',
  'формат',
  'для',
  'сумочка'],
 ['визуально', 'всё', 'хорошо', 'не', 'пробовать', 'купить', 'в', 'подарок'],
 ['коробка',
  'в',
  'состояние',
  'пройти',
  'афган',
  'гладкий',
  'кожа',
  'носок',
  'сапог',
  'тоже',
  'уже',
  'не',
  'выглядеть',
  'новый',
  'но',
  'сапог',
  'вернуть',
  'совсем',
  'не',
  'по',
  'это',
  'причина',
  'дело',
  'в',
  'тот',
  'что',
  'всама',
  'модель',
  'сапог',
  'странный',
  'голенище',
  'дизайнер',
  'просчитаться',
  'решить',
  'вставить',